### Importando as bibliotecas necessarias para o ETL

In [112]:
import os
import json
import pandas as pd
import openpyxl

### 1. Definição dos Caminhos dos Arquivos
### Especificamos os caminhos absolutos dos arquivos de entrada

In [113]:
caminho_json = "C:/Users/win 10/Documents/GitHub/neotass_etl/data/registros_oportunidades.json"
caminho_parquet = "C:/Users/win 10/Documents/GitHub/neotass_etl/data/sellout.parquet"

### 2. Carregamento do Arquivo JSON
### Verificamos se o arquivo existe antes de tentar abri-lo

In [114]:
if os.path.exists(caminho_json):
    with open(caminho_json, "r", encoding="utf-8") as file:
        oportunidades = json.load(file)
    
    # Convertendo os dados carregados em um DataFrame do Pandas
    df_oportunidades = pd.DataFrame(oportunidades)
    
    # Exibir as primeiras linhas do DataFrame para conferência
    display(df_oportunidades.head())
else:
    print(f"Erro: Arquivo não encontrado no caminho {caminho_json}")

,Data de Registro,Quantidade,Status,Nome Fantasia,CNPJ Parceiro,Telefone Parceiro,Categoria produto,Nome_Produto,Valor_Unitario
0,1677283200000,23,Fechada,Marques Ferreira Ltda.,71.956.320/0001-68,71 8373-4031,Tablet,LiteTab Pro,2871.09
1,1712707200000,28,Perdida,Novaes,52.346.709/0001-47,0800-268-6025,Tablet,QuantumPad Ultra,2854.77
2,1709078400000,35,Perdida,Cirino,10.789.432/0001-87,41 9524 9911,Tablet,InfinityTab X,3550.90
3,1685836800000,25,Aberta,Novais,12.403.859/0001-20,0900 376 1847,Tablet,InfinityTab s,572.16
4,1729900800000,90,Fechada,Andrade,15.863.492/0001-16,(071) 2090 4217,Business TV,Collaboration Edge,4685.77


### 3. Carregamento do Arquivo Parquet
### Verificamos se o arquivo existe antes de tentar carregá-lo

In [115]:
if os.path.exists(caminho_parquet):
    df_sellout = pd.read_parquet(caminho_parquet)
    
    # Exibir as primeiras linhas do DataFrame para conferência
    display(df_sellout.head())
else:
    print(f"Erro: Arquivo não encontrado no caminho {caminho_parquet}")

,Data_Fatura,Quantidade,NF,Nome Fantasia,CNpj Parceiro,Telefone,Categoria,Nome_Produto,Valor_Unitario
0,2023-06-19,98,NF_267049,Pastor - EI,54.318.792/0001-76,(041) 4384-2150,Business TV,Collaboration Edge,4685.77
1,2023-03-30,85,NF_291155,Novaes - ME,85.730.216/0001-65,81 1706 3048,Smartphone,Velocity Elite,1878.94
2,2024-02-18,97,NF_907848,Nascimento,12.064.379/0001-82,(061) 5904 2597,Monitor,InfinityScreen Ultra,2639.11
3,2024-12-26,8,NF_600759,Nogueira,31.287.095/0001-08,+55 71 4014 0119,Monitor,PixelMaster S,4455.39
4,2023-06-26,34,NF_821240,Sampaio,89.231.756/0001-55,+55 51 7548 2145,Business TV,BizScreen S,4067.43


### 4. Transformação dos Dados
### 4.1 Renomeando colunas do DataFrame de Oportunidades
### Ajustamos os nomes das colunas para um formato mais padronizado

In [116]:
df_oportunidades.rename(columns={
    "id": "id_oportunidade",
    "partner_id": "id_parceiro",
    "product_id": "id_produto",
    "Data de Registro": "Data_Registro",
    "amount": "Quantidade",
    "$": "Valor Unitario",
    "status": "status",
    "Categoria produto": "Categoria_Produto",
    "Telefone Parceiro": "Telefone_Parceiro",
    "Nome Fantasia": "Nome_Fantasia",
}, inplace=True)

# Convertendo a coluna de data para o formato datetime
df_oportunidades["Data_Registro"] = pd.to_datetime(df_oportunidades["Data_Registro"]).dt.normalize()

# Convertendo a coluna de valor com duas casas decimais
df_oportunidades["Valor_Unitario"] = df_oportunidades["Valor_Unitario"].astype(float).round(2)

# Remoção de Caratecteres especiais das colunas CNPJ e Telefone
df_oportunidades["CNPJ Parceiro"] = df_oportunidades["CNPJ Parceiro"].astype(str).str.replace(r'\D', '', regex=True)
df_oportunidades["Telefone_Parceiro"] = df_oportunidades["Telefone_Parceiro"].astype(str).str.replace(r'\D', '', regex=True)

# Exibir as primeiras linhas para conferência
display(df_oportunidades.head())

,Data_Registro,Quantidade,Status,Nome_Fantasia,CNPJ Parceiro,Telefone_Parceiro,Categoria_Produto,Nome_Produto,Valor_Unitario
0,1970-01-01,23,Fechada,Marques Ferreira Ltda.,71956320000168,7183734031,Tablet,LiteTab Pro,2871.09
1,1970-01-01,28,Perdida,Novaes,52346709000147,08002686025,Tablet,QuantumPad Ultra,2854.77
2,1970-01-01,35,Perdida,Cirino,10789432000187,4195249911,Tablet,InfinityTab X,3550.90
3,1970-01-01,25,Aberta,Novais,12403859000120,09003761847,Tablet,InfinityTab s,572.16
4,1970-01-01,90,Fechada,Andrade,15863492000116,07120904217,Business TV,Collaboration Edge,4685.77


In [117]:
#print(df_oportunidades.columns)
print(df_sellout.columns)


Index(['Data_Fatura', 'Quantidade', 'NF', 'Nome Fantasia', 'CNpj Parceiro',
       'Telefone', 'Categoria', 'Nome_Produto', 'Valor_Unitario'],
      dtype='object')


### 4.2 Renomeando colunas do DataFrame de Sellout
### Ajustamos os nomes das colunas para um formato mais padronizado

In [118]:
df_sellout.rename(columns={
    "id": "id_sellout",
    "partner_id": "id_parceiro",
    "product_id": "id_produto",
    "Data_Fatura": "data_fatura",
    "invoice_number": "nf",
    "quantity": "quantidade",
    "total_value": "valor_total",
    "CNpj Parceiro": "CNPJ_Parceiro",
}, inplace=True)

# Convertendo a coluna de data para datetime
df_sellout["data_fatura"] = pd.to_datetime(df_sellout["data_fatura"])

# Remoção de Caracteres especiais das colunas CNPJ e Telefone
df_sellout["CNPJ_Parceiro"] = df_sellout["CNPJ_Parceiro"].astype(str).str.replace(r'\D', '', regex=True)
df_sellout["Telefone"] = df_sellout["Telefone"].astype(str).str.replace(r'\D', '', regex=True)

# Exibir as primeiras linhas para conferência
display(df_sellout.head())

,data_fatura,Quantidade,NF,Nome Fantasia,CNPJ_Parceiro,Telefone,Categoria,Nome_Produto,Valor_Unitario
0,2023-06-19,98,NF_267049,Pastor - EI,54318792000176,04143842150,Business TV,Collaboration Edge,4685.77
1,2023-03-30,85,NF_291155,Novaes - ME,85730216000165,8117063048,Smartphone,Velocity Elite,1878.94
2,2024-02-18,97,NF_907848,Nascimento,12064379000182,06159042597,Monitor,InfinityScreen Ultra,2639.11
3,2024-12-26,8,NF_600759,Nogueira,31287095000108,557140140119,Monitor,PixelMaster S,4455.39
4,2023-06-26,34,NF_821240,Sampaio,89231756000155,555175482145,Business TV,BizScreen S,4067.43


### 5. Carregamento dos Dados
### Criamos um diretório para salvar os arquivos transformados

In [119]:
# Criamos um diretório para salvar os arquivos transformados
output_dir = "C:/Users/win 10/Documents/GitHub/neotass_etl/output"
os.makedirs(output_dir, exist_ok=True)

# Salvamos os DataFrames no formato Excel
df_oportunidades.to_excel(os.path.join(output_dir, "fato_registro_oportunidade.xlsx"), index=False)
df_sellout.to_excel(os.path.join(output_dir,"fato_sellout.xlsx"), index=False)

print("ETL concluído com sucesso! Os arquivos foram salvos na pasta 'output'.")

ETL concluído com sucesso! Os arquivos foram salvos na pasta 'output'.
